## 시계열 데이터 다루기

자전거 시스템의 시간대별 자전거 대여 데이터, 날씨와 계절 정보까지 포함된 데이터셋.


소스 데이터에서 각 행은 시간별로 분리된 데이터다. 시간 행으로 되어 있는 구성을 바꿔 인덱스를 올릴 때마다 일자가 바뀌고 또 다른 축에서는 하루의 시간을 표현하게 만들자. 나머지 축에는 데이터의 각 열에 해당하는 날씨와 기온 등의 데이터가 포함된다.

In [1]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, threshold=50, linewidth=75)

In [2]:
bikes_numpy = np.loadtxt(
    "/content/drive/MyDrive/deep_learning_with_pytorch/data/p1ch4/bike-sharing-dataset/hour-fixed.csv", 
    dtype=np.float32, 
    delimiter=",", 
    skiprows=1, 
    converters={1: lambda x: float(x[8:10])}) # <1>
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
        [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
        ...,
        [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
        [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]])

매 시간대별 데이터셋의 정보는 다음과 같다.


- 레코드 인덱스 : instant
- 일자 : day
- 계절 : season
- 연도 : yr
- 월 : mnth
- 시간 : hr
- 공휴일 상태 : holiday
- 요일 : weekday
- 평일 상태 : workingday
- 날씨 상태 : weathersit (1: 맑음, 2: 안개, 3: 약한 비/눈, 4: 강한 비/눈)
- 섭씨 기온 : temp
- 체감 섭씨 기온 : atemp
- 습도 : hum
- 풍속 : windspeed
- 일반 사용자 : casual 
- 등록 사용자 : registered
- 대여 자전거 수 : cnt

## 시간 단위로 데이터 만들기

일별로 매 시간의 데이터셋을 구하기 위해 동일 텐서를 24시간 배치로 바라보는 뷰가 필요하다. bieks 텐서의 차원 정보(shape)와 스트라이드(stride)를 확인하자.

In [4]:
bikes.shape, bikes.stride()

#17,520시간에 17개 열이다. 이제 데이터를 일자, 시간, 17개 열의 세 개 축으로 만들자.

(torch.Size([17520, 17]), (17, 1))

In [5]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

# bikes.shape[1]은 bikes 텐서의 열의 개수로 17이다.
# -1은 '남은 차원과 거기에 들어있는 요소를 다 합친 것'을 의미하는 placeholder로 사용한다.

(torch.Size([730, 24, 17]), (408, 17, 1))

daily_bikes에 대한 스트라이드를 보면 두 번째 차원인 시간을 한 칸 이동하려면 저장 공간상에서 열의 개수인 17칸만큼 이동해야 하고 첫 번째 차원인 일자를 한 칸 이동하려면 한 행의 길이에 24를 곱한 수, 즉 이 예에서는 17 * 24 = 408이 된다.


가장 오른쪽 차원은 원래 데이터셋에서와 같이 열의 수가 된다. 가운데 차원은 연속된 24시간을 나타내는 시간 차원이다. 표현을 달리하면 C개의 채널을 가진 하루를 L시간으로 나눈 N개의 연속된 값을 가진다고 볼 때, 이를 N* C * L 순서로 놓고 싶다면 텐서를 전치해야 한다.

In [6]:
daily_bikes = daily_bikes.transpose(1,2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

## 훈련 준비

'날씨 상태'를 나타내는 데이터는 순서값이다. 총 4단계 = 1~4로 좋은 날씨 순이다. 이 값을 카테고리로 간주해 각 단계를 레이블로 볼 수 있고 동시에 연속값으로 볼 수도 있다. 카테고리로 보려면 값을 원핫 인코딩 벡터로 만들어 데이터셋과 병합해야 한다.

In [7]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day

tensor([[ 1,  1,  ..., 13, 16],
        [ 2,  1,  ..., 32, 40],
        ...,
        [23,  1,  ..., 17, 28],
        [24,  1,  ..., 24, 39]])

In [8]:
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2,
        2, 2])

그리고 날씨 수준에 따라 행렬을 원핫 인코딩으로 변환하자. 이전 절에서 했던 것처럼 unsqueeze를 사용하여 싱글톤 차원을 더해준다.

In [9]:
weather_onehot.scatter_(
    dim=1,
    index=first_day[:,9].unsqueeze(1).long() - 1, #날씨값은 1부터 4까지지만, 색인은 0에서 시작하므로 1을 빼줌.
    value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [10]:
#마지막으로 만들어진 행렬을 원래 데이터셋에 cat 함수를 사용해 병합한다. 결과에서 첫 부분을 살펴보자.
torch.cat((bikes[:24], weather_onehot), 1)[:1]

#뒤에 네 열이 1,0,0,0 확인

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          6.0000,  0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,
          3.0000, 13.0000, 16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

이제 원래의 bikes 데이터셋에 성공적으로 '날씨 상태'를 원핫 인코딩하여 데이터 열이 있는 차원을 따라 병합했다. 

바꿔 말하면 두 개의 데이터셋 열이 겹겹이 쌓이게 됐다고 볼 수 있으며, 원래의 데이터셋 뒤에 새로운 원핫 인코딩 열이 붙었다고 생각할 수도 있다.

daily_bikes 텐서에 대해서도 동일한 작업을 할 수 있다. 이 데이터는 (B,C,L) 형태이고 L = 24임을 기억하자. 일단 값이 0이고 크기가 각각 B와 L, 그리고 추가 열의 크기는 C인 텐서를 만들자.

In [11]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])

daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [12]:
daily_weather_onehot.scatter_(
    1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)

daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [13]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

In [14]:
#값을 변환해서 0.0을 1.0으로 바꿀 수도 있다.

daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0

In [15]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)
                         / (temp_max - temp_min))

In [16]:
#혹은 다음과 같이 모든 값에서 평균을 빼고 표준편차로 나누기도 한다.
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp))
                         / torch.std(temp))